In [2]:
import logging

from pathlib import Path

from Pegasus.api import *

logging.basicConfig(level=logging.DEBUG)

# --- Replicas -----------------------------------------------------------------
with open("f.a", "w") as f:
    f.write("This is sample input to KEG")

fa = File("f.a").add_metadata(creator="ryan")
rc = ReplicaCatalog().add_replica("local", fa, Path(".") / "f.a")

# --- Transformations ----------------------------------------------------------
preprocess = Transformation(
                "preprocess",
                site="condorpool",
                pfn="/home/scitech/pegasus/dist/pegasus-5.0.0dev/bin/pegasus-keg",
                is_stageable=False,
                arch=Arch.X86_64,
                os_type=OS.LINUX
            )

findrange = Transformation(
                "findrange",
                site="condorpool",
                pfn="/home/scitech/pegasus/dist/pegasus-5.0.0dev/bin/pegasus-keg",
                is_stageable=False,
                arch=Arch.X86_64,
                os_type=OS.LINUX
            )

analyze = Transformation(
                "analyze",
                site="condorpool",
                pfn="/home/scitech/pegasus/dist/pegasus-5.0.0dev/bin/pegasus-keg",
                is_stageable=False,
                arch=Arch.X86_64,
                os_type=OS.LINUX
            )

tc = TransformationCatalog().add_transformations(preprocess, findrange, analyze)

# --- Workflow -----------------------------------------------------------------
'''
                     [f.b1] - (findrange) - [f.c1] 
                    /                             \
[f.a] - (preprocess)                               (analyze) - [f.d]
                    \                             /
                     [f.b2] - (findrange) - [f.c2]

'''
wf = Workflow("blackdiamond")

fb1 = File("f.b1")
fb2 = File("f.b2")
job_preprocess = Job(preprocess)\
                    .add_args("-a", "preprocess", "-T", "3", "-i", fa, "-o", fb1, fb2)\
                    .add_inputs(fa)\
                    .add_outputs(fb1, fb2)

fc1 = File("f.c1")
job_findrange_1 = Job(findrange)\
                    .add_args("-a", "findrange", "-T", "3", "-i", fb1, "-o", fc1)\
                    .add_inputs(fb1)\
                    .add_outputs(fc1)

fc2 = File("f.c2")
job_findrange_2 = Job(findrange)\
                    .add_args("-a", "findrange", "-T", "3", "-i", fb2, "-o", fc2)\
                    .add_inputs(fb2)\
                    .add_outputs(fc2)

fd = File("f.d")
job_analyze = Job(analyze)\
                .add_args("-a", "analyze", "-T", "3", "-i", fc1, fc2, "-o", fd)\
                .add_inputs(fc1, fc2)\
                .add_outputs(fd)

wf.add_jobs(job_preprocess, job_findrange_1, job_findrange_2, job_analyze)
wf.add_replica_catalog(rc)
wf.add_transformation_catalog(tc)

try:
    wf.plan(submit=True)\
        .wait()\
        .analyze()\
        .statistics()
except PegasusClientError as e:
    print(e.output)


Plan:
2020.06.27 04:06:12.046 UTC:    
2020.06.27 04:06:12.052 UTC:   ----------------------------------------------------------------------- 
2020.06.27 04:06:12.057 UTC:   File for submitting this DAG to HTCondor           : blackdiamond-0.dag.condor.sub 
2020.06.27 04:06:12.062 UTC:   Log of DAGMan debugging messages                 : blackdiamond-0.dag.dagman.out 
2020.06.27 04:06:12.069 UTC:   Log of HTCondor library output                     : blackdiamond-0.dag.lib.out 
2020.06.27 04:06:12.074 UTC:   Log of HTCondor library error messages             : blackdiamond-0.dag.lib.err 
2020.06.27 04:06:12.080 UTC:   Log of the life of condor_dagman itself          : blackdiamond-0.dag.dagman.log 
2020.06.27 04:06:12.085 UTC:    
2020.06.27 04:06:12.090 UTC:   -no_submit given, not submitting DAG to HTCondor.  You can do this with: 
2020.06.27 04:06:12.101 UTC:   ----------------------------------------------------------------------- 
2020.06.27 04:06:12.941 UTC:   Your database is co

[##################################################] 100.0% ..Success (Completed: 12, Queued: 0, Running: 0, Failed: 0)


Analyzer:
Your database is compatible with Pegasus version: 5.0.0dev

************************************Summary*************************************

 Submit Directory   : /home/scitech/shared-data/scitech/pegasus/blackdiamond/run0004
 Total jobs         :     12 (100.00%)
 # jobs succeeded   :     12 (100.00%)
 # jobs failed      :      0 (0.00%)
 # jobs held        :      0 (0.00%)
 # jobs unsubmitted :      0 (0.00%)

 

Statistics:
Your database is compatible with Pegasus version: 5.0.0dev

#
# Pegasus Workflow Management System - http://pegasus.isi.edu
#
# Workflow summary:
#   Summary of the workflow execution. It shows total
#   tasks/jobs/sub workflows run, how many succeeded/failed etc.
#   In case of hierarchical workflow the calculation shows the
#   statistics across all the sub workflows.It shows the following
#   statistics about tasks, jobs and sub workflows.
#     * Succeeded - total count of succeeded tasks/jobs/sub workflows.
#     * Failed - total count of failed t